<a href="https://colab.research.google.com/github/bxck75/piss-ant-pix2pix/blob/master/modeltransferv2_pailkids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# remove defaults
!rm -r sample_data
# Clone the repo
!git clone https://github.com/bxck75/Python_Helpers.git

# Change dir
%cd /content/Python_Helpers
# install
!python setup.py install

from IPython.display import clear_output
import main
import os
import sys
import IPython
import Helpers
P=Helpers.core.Core()
hlp=P.H
clear_output()

In [0]:
# get repo
%cd /content/
hlp.repolist= hlp.repo_collection
repos_sorted = hlp.repo_collection.repos_sorted
A1=['bxck75/piss-ant-pix2pix']
hlp.Me(['inst_reps',A1,'/content/installed_repos',False,True])
clear_output()

In [0]:
# get images
%cd /content/
# images_set_name='insect'
# images_set_code='1fbEUcBIvLwjlVFrUiW2g8KtRTvFSsW9D'
# images_model_code='1eFR5ZHfp5PzPEnFv_v3xxE9T0tKPJTS4'

# https://drive.google.com/open?id=1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv # images
# https://drive.google.com/open?id=1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9

# images_set_name='apes'
# images_set_code='1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv'
# images_model_code='1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9'

# https://drive.google.com/open?id=1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-
images_set_name='garbagepailkids'
images_set_code='1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-'
images_model_code='1tp8Gm1fMuSQoLEEXQ_C1Dlw1N4wwFxKb'

Helpers.GdriveD.GdriveD(images_set_code, '/content/'+images_set_name+'.zip')
!unzip /content/{images_set_name}.zip
clear_output()

In [0]:
%%writefile /content/installed_repos/piss-ant-pix2pix/image_folder_make_set_train_colab_450.sh
#!/bin/bash
##########################################################################
#                                                                        #
#									 #

#
#
##########################################################################
# from folder to pix2pix set and train/test
# usage :./image_folder_make_set_train.sh <images folder> <epochs to train> <prep/train/test/lucky-triple>
# bash image_folder_make_set_train.sh $MYHOME/data/steenmarter 5
# it wil make : 
#  -- <image_folder>_resize (resized pix2pix training set)
#  -- <image_folder>_train (training metrics) beside the original folder

# index.html in the train folder has the evaluation of the training and test

# set root tmp folder
root='/content/'
input_img_folder=${1}
training_epochs=${2}
prep=${3} #  True or False wil switch prepping off if this is already done
root_tmp_folder='p2p_process_tmp'
root_data_folder='/root/Bureaublad/data/'
TF_CPP_MIN_LOG_LEVEL='3'  # or any {'0', '1', '2'}'

function process {
	#remove old tmp files if any...
# 	sudo rm -r ${root}${root_tmp_folder}
	# Resize source images
	python tools/process.py --input_dir ${input_img_folder} --operation resize --pad --size 450 --output_dir ${root}${root_tmp_folder}/_resized


	# Create images with blank centers
	# python tools/process.py --input_dir ${root}${root_tmp_folder}/_resized --operation blank --output_dir ${root}${root_tmp_folder}/_blank
	# Create images with edges
	sudo mkdir -p ${root}${root_tmp_folder}/_edges
	python tools/edge.py --input_dir ${root}${root_tmp_folder}/_resized --output_dir ${root}${root_tmp_folder}/_edges
	ls ${root}${root_tmp_folder}/_edges


	# Combine resized images with blanked images side by side
	# python tools/process.py --input_dir ${root}${root_tmp_folder}/_resized --b_dir  ${root}${root_tmp_folder}/_blank --operation combine --output_dir  ${root}${root_tmp_folder}/_resized/_combined
	
	# Combine resized images with edge images side by side
	python tools/process.py --input_dir ${root}${root_tmp_folder}/_resized --b_dir  ${root}${root_tmp_folder}/_edges --operation combine --output_dir  ${root}${root_tmp_folder}/_resized/_combined
	
	# Split into train/val set
	python tools/split.py --dir ${root}${root_tmp_folder}/_resized/_combined
	# make dir and copy final images
	sudo mkdir -p ${input_img_folder}_resize
    sudo mkdir ${input_img_folder}images
	cp -rf ${root}${root_tmp_folder}/_resized/_combined/ ${input_img_folder}images/
}

function train {
	# Training pix2pix
	x=$1
	echo "[Start training....]"
	# xterm_sizes="55x10+0+${x}"
# 	xterm_style="-misc-fixed-medium-r-normal--10-*-*-*-*-*-iso8859-15"
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "tensorboard --port 6001 --logdir=${input_img_folder}_train" & 
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "chromium --no-sandbox http://debian:6001" &
	# ((x+=185))
# 	xterm_sizes="55x35+0+${x}"
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "python pix2pix.py --mode train --output_dir ${input_img_folder}_train --max_epochs ${training_epochs} --input_dir ${input_img_folder}_resize/_combined/train --which_direction BtoA" &
    python pix2pix.py --mode train --output_dir ${input_img_folder}_train --max_epochs ${training_epochs} --input_dir ${input_img_folder}_resize/_combined/train --which_direction BtoA

}

function test {
	#Testing model trained so far
	x=$1
	echo "[Start testing....]"
# 	xterm_style="-misc-fixed-medium-r-normal--10-*-*-*-*-*-iso8859-15"
# 	xterm_sizes="55x55+335+${x}"
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "tensorboard --port 6002 --logdir=${input_img_folder}_train" & disown
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "chromium --no-sandbox http://debian:6002" &
# 	# ((x+=185))
# 	xterm_sizes="55x55+335+${x}"
# 	xterm -fn $xterm_style -geometry $xterm_sizes -hold -e "python pix2pix.py --mode test --output_dir ${input_img_folder}_test --input_dir ${input_img_folder}_resize/_combined/val --checkpoint ${input_img_folder}_train" &
    python pix2pix.py --mode test --output_dir ${input_img_folder}_test --input_dir ${input_img_folder}_resize/_combined/val --checkpoint ${input_img_folder}_train

}

function show_t_t {
	# Show Training summary html
	xterm -geometry "5x5x5x5" -e "chromium --no-sandbox ${input_img_folder}_train/index.html" & disown 
	# Show Testing summary html
	xterm -geometry "5x5x5x5" -e "chromium --no-sandbox ${input_img_folder}_test/index.html" & disown
}

function clean {
	#remove tmp files
	sudo rm -r ${root}${root_tmp_folder}
}


# Workflow sequence 
if [[ $prep == 'prep' ]] || [[ $prep == 'lucky-triple' ]]; then
	process
fi
wait
if [[ $prep == 'train' ]] || [[ $prep == 'lucky-triple' ]]; then
	train 1000
fi
wait
if [[ $prep == 'test' ]] || [[ $prep == 'lucky-triple' ]]; then
	test 1000
fi
wait
# clean


In [0]:
# make dataset
%cd /content/installed_repos/piss-ant-pix2pix
!bash image_folder_make_set_train_colab_450.sh /content/{images_set_name} 1 prep
clear_output()

In [0]:
# ''' 
#     startup loop  1x
#     # 1 get metric from gdrive
#     2 train 5 epochs
#     3 dump metrics to gdrive with the same file id 
# '''
# for i in range(1):
#     # fetch metrics
#     %cd /content/
#     # Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
#     # !unzip /content/{images_set_name}_model_in.zip -d metrics/
#     # !rm -r /content/metrics/options.json
#     # train epochs
#     %cd /content/installed_repos/piss-ant-pix2pix
#     !python pix2pix.py --output_dir /content/metrics --progress_freq 100 --save_freq 150 --summary_freq 100 --display_freq 100 --max_epochs 1 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'
#     # dump metrics
#     %cd /content/
#     Helpers.Core()
#     obj=Helpers.core.Core()
#     folder_of_model='metrics'
#     result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
#     zip_hash=result.split('(id) ')[1]
#     print(zip_hash)
#     # delete old crap
#     !rm -r /content/metrics
#     !rm -r /content/{images_set_name}model.zip

start values
progress  epoch 1  step 10  image/sec 0.7  remaining 122m
dloss_GAN 0.11989411
gloss_GAN 0.060013566
gloss_L1 0.043864187

In [0]:
''' 
    loop  10x
    1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
for i in range(10):
    # fetch metrics
    %cd /content/
    Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    !unzip /content/{images_set_name}_model_in.zip -d metrics/
    !rm -r /content/metrics/options.json
    # train epochs
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --checkpoint /content/metrics --output_dir /content/metrics --progress_freq 10 --save_freq 50 --summary_freq 50 --display_freq 200 --max_epochs 5 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'

    %cd /content/
    '''clean old models,test images,logs etc befor zipping'''
    # delete unwanted
    !rm -r /content/metrics/images    
    !rm -r /content/metrics/index.html
    # clean up models
    models_list=hlp.Me(['globx','/content/metrics','model-*'])
    models_list = sorted(models_list)
    print(models_list)
    n=3 #pop the last 3 items off the list (latest model)
    del models_list[-n:]
    # delete the files left in the list
    for m_file in models_list:
        print('deleting : ' + m_file)
        !rm -r {m_file}
    # clean up events
    events_list=hlp.Me(['globx','/content/metrics','events*'])
    events_list = sorted(events_list)
    print(events_list)
    n=1 #pop the last item off the list (latest event) 
    del events_list[-n:]
    # delete the files left in the list
    for e_file in events_list:
        print('deleting : ' + e_file)
        !rm -r {e_file}

    '''zip metrrics to gdrive'''
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)
    # delete old crap
    !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

In [0]:
# # rewrite checkpointfile
# !echo 'model_checkpoint_path: "/content/metrics/model-21880"' > /content/metrics/checkpoint
# !echo 'all_model_checkpoint_paths: "/content/metrics/model-21880"' >> /content/metrics/checkpoint

In [0]:
# download/unzip metrics 
# Get pretrained metrics
%cd /content/
Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
!unzip /content/{images_set_name}_model_in.zip -d metrics/

In [0]:
# # remove old crap before rezipping
# !rm -r /content/metrics/images
# !rm -r /content/metrics/events.out.tfevents*
# !rm -r /content/metrics/index.html
hlp.Me(['globx','/content/metrics','model-*'])

In [0]:
# # rezip
# %cd /content/
# Helpers.Core()
# obj=Helpers.core.Core()
# folder_of_model='metrics'
# print(obj.Gdrive_root+'/models')
# result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# # print(result)
# zip_hash=result.split('(id) ')[1]
# print(zip_hash)

In [0]:
# test the current checkpoint
# get metrics
%cd /content/
Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
!unzip /content/{images_set_name}_model_in.zip -d metrics/
# test 
%cd /content/installed_repos/piss-ant-pix2pix
!python pix2pix.py --mode test --output_dir /content/{images_set_name}_pretrained/test --input_dir /content/{images_set_name}images/_combined/val --max_steps 100 --which_direction "BtoA" --seed 0 --checkpoint /content/metrics
# export
!python pix2pix.py --mode export --output_dir /content/{images_set_name}_pretrained/exp --checkpoint /content/metrics


In [0]:

# rezip clean export
%cd /content/
Helpers.Core()
obj=Helpers.core.Core()
folder_of_model=images_set_name+'_pretrained/exp'
print(obj.Gdrive_root+'/models')
result=obj.H.zip(images_set_name+'model_exp',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# print(result)
zip_hash=result.split('(id) ')[1]
print(zip_hash)

In [0]:
# test exported path
%cd /content/installed_repos/piss-ant-pix2pix
!python pix2pix.py --mode test --output_dir /content/{images_set_name}_pretrained/exp_test --input_dir /content/{images_set_name}images/_combined/val --checkpoint /content/garbagepailkids_pretrained_test --max_steps 100 --which_direction "BtoA" --seed 0 --checkpoint /content/metrics
